In [2]:
import pandas as pd
import mysql.connector
import nltk
from textblob import TextBlob
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.decomposition import LatentDirichletAllocation
import textstat

from nltk.stem import PorterStemmer


# Feature Enginnering

loud datasets

In [3]:
def read_data_from_mysql(table_name):
    # 设置本地MySQL数据库连接信息
    host = 'localhost'  # 替换为本地MySQL数据库的主机名
    user = 'root'  # 替换为数据库用户名
    password = '991208louise'  # 替换为数据库密码
    database = 'amazon'  # 替换为数据库名称

    # 建立数据库连接
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    # Read data from MySQL into a DataFrame
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql(query, conn)
    print(f'load {table_name} from MySQL!')
    # df.drop(columns=['reviewerID'], inplace=True)

    # Close the database connection
    conn.close()

    return df

In [4]:
def saved_mysql(df, table_name):
    # 设置本地MySQL数据库连接信息
    host = 'localhost'  # 替换为本地MySQL数据库的主机名
    user = 'root'  # 替换为数据库用户名
    password = '991208louise'  # 替换为数据库密码
    database = 'amazon'  # 替换为数据库名称

    # 建立数据库连接
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    # 检查表是否存在
    table_exists = False
    with conn.cursor() as cursor:
        check_table_sql = f"SHOW TABLES LIKE '{table_name}';"
        cursor.execute(check_table_sql)
        table_exists = cursor.fetchone() is not None
        print(f"Table '{table_name}' exists: {table_exists}")
    if table_exists:
        print(f"Table '{table_name}' already exists. Drop it...")
        with conn.cursor() as cursor:
            # Move the del_table query execution inside the 'with' block
            del_table = f"DROP TABLE {table_name};"
            cursor.execute(del_table)
            # table_exists 设置为反面
            table_exists=False

    if not table_exists:
        print(f"Table '{table_name}' does not exist. Creating new table...")
        column_types = {
            'helpfulness_score': 'FLOAT',
            'reviewText': 'LONGTEXT',
            'cleaned_review': 'LONGTEXT',
            'overall': 'FLOAT',
            'review_length': 'INT',
            'num_sentences': 'INT',
            'avg_sentence_length': 'FLOAT',
            'subjectivity': 'FLOAT',
            'readability': 'FLOAT',
            'timeline': 'INT',
            'topic':'int',
            'review_volume':'INT',
        }



        # 构建创建表的SQL语句
        create_table_sql = f"CREATE TABLE {table_name} ("
        for col_name, col_type in column_types.items():
            create_table_sql += f"{col_name} {col_type}, "
        create_table_sql = create_table_sql.rstrip(', ')  # 去除最后一个逗号和空格
        create_table_sql += ");"

        # 创建表
        with conn.cursor() as cursor:
            cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created.")

    df.drop(columns=['reviewerID','asin','helpful_vote','total_vote'], inplace=True)


    # 将DataFrame数据插入数据库表
    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"
            cursor.execute(insert_sql, tuple(row))

    # 提交数据
    conn.commit()
    print(f'Finished saving {table_name} to MySQL!')
    # 关闭数据库连接
    conn.close()


# **Content Features**

In [4]:
def preprocess_text(text):
    # Tokenize text
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Remove punctuation from within words
    words = [word.translate(str.maketrans('', '', string.punctuation)) for word in words]
    # lower capitalization
    words = [word.lower() for word in words]


    # Stem words
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    # Remove words with length less than 3
    words = [word for word in words if len(word) >= 3]

    return ' '.join(words)



def clean_text(df):
    df['cleaned_review'] = df['reviewText'].apply(preprocess_text)
    # print(df['cleaned_review'].head())
    print('\nText cleaned!')
    return df

Review length


In [5]:
def review_length(df):
    df['review_length']=df['reviewText'].apply(words_count)
    print('\nReview length extracted!')
    return df
def words_count(text):
    # Tokenize text
    words = word_tokenize(text)
    return len(words)

Number of Sentence

In [6]:
# Function to count sentences in each review
def count_sentences(review):
    sentences = nltk.sent_tokenize(review)
    return len(sentences)
def num_sentences(df):
    df['num_sentences']=df['reviewText'].apply(count_sentences)
    print('\nNumber of sentences extracted!')
    return df

Average Sentence length

In [7]:
def avg_sentence_length(df):
    # print(df['num_sentences'])
    df['avg_sentence_length']=df['review_length']/df['num_sentences']
    print('\nAverage sentence length extracted!')
    return df

Subjective

In [8]:
def get_subjective(review):
    blob = TextBlob(review)
    subjectivity = blob.sentiment.subjectivity
    return subjectivity

def subjective(df):
    df['subjectivity'] = df['reviewText'].apply(get_subjective)
    print('\nSubjective extracted!')
    return df


Readability

In [9]:
def readability(df):
    # Calculate SMOG Index for each review and store in a new column 'smog_index'
    df['readability'] = df['reviewText'].apply(lambda x: textstat.smog_index(x))
    # Print the DataFrame with the SMOG Index
    print('\nReadability extracted!')
    return df

Timeline

In [10]:

# 将reviewTime列转换为日期时间类型
def timeline(df):
    df['reviewTime'] = pd.to_datetime(df['reviewTime'])
    # 设置目标日期为2014年7月31日
    target_date = pd.to_datetime('2014-07-31')
    # 计算每个日期距离目标日期的天数，并将结果保存在新的列'days_before_target'中
    df['timeline'] = (target_date - df['reviewTime']).dt.days
    df.drop(columns=['reviewTime'], inplace=True)
    print('\nTimeline extracted!')
     # 作图显示天数的分布
    # plt.hist(df['timeline'], bins=20, edgecolor='black')
    # plt.xlabel('Days before 2014-07-31')
    # plt.ylabel('Number of Reviews')
    # plt.title('Distribution of Reviews Before 2014-07-31')
    # plt.show()
    return df

Lexical Features

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_features=500, max_df=0.95,min_df=0.1, stop_words='english',)
def TFIDF(df):
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_review'])
    feature_names=tfidf_vectorizer.get_feature_names_out()
    # feature_names = np.array(feature_names)
    feature_names.tolist()
    # 将list保存为txt文件
    with open('results/feature_names.txt', 'a') as f:
        for item in feature_names:
            f.write("%s\n" % item)

    return tfidf_matrix

# def Lexical(df):
#     # 将文本数据转换为TF-IDF矩阵
#     tfidf_matrix = TFIDF(df)
#     # 将计算得到的TF-IDF矩阵保存到新列'lexical'中
#     df['lexical'] = [row for row in tfidf_matrix.toarray()]
#     print('\nLexical features extracted!')
#
#     return df

latent review topics distribution

In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

# 假设您的数据存储在一个名为df的DataFrame中，包含一个名为'cleaned_review'的列
def topics(df):
    tfidf=TFIDF(df)
    print('tfidf get')
    # 使用NMF模型提取主题
    num_topics = 10
    nmf_model = NMF(n_components=num_topics, random_state=42)
    print('nmf_model get')
    nmf_topic_matrix = nmf_model.fit_transform(tfidf)
    print('nmf_topic_matrix get')

    # 规范化主题矩阵
    nmf_topic_matrix = normalize(nmf_topic_matrix, norm='l1', axis=1)
    print('nmf_topic_matrix normalize')

    # 提取主题关键词
    feature_names = tfidf_vectorizer.get_feature_names_out()
    topic_keywords = []
    for topic_idx, topic in enumerate(nmf_model.components_):
        top_keywords_idx = topic.argsort()[-10:][::-1]
        top_keywords = [feature_names[i] for i in top_keywords_idx]
        topic_keywords.append(top_keywords)

    # 保存主题关键词到txt文件
    with open('results/topics_keywords.txt', 'a') as f:
        for topic_idx, keywords in enumerate(topic_keywords):
            f.write(f"Topic {topic_idx + 1}: {', '.join(keywords)}\n")

    # 提取可能性最大的主题序号
    max_topic_indices = np.argmax(nmf_topic_matrix, axis=1)
    df['topic'] = max_topic_indices

    print('\nTopics extracted:')
    print('df[topic] get:', df['topic'].head())
    return df



review volume

In [13]:
def review_volume(df):
    # Calculate review volume for each product
    df['review_volume'] = df.groupby('asin')['asin'].transform('count')
    print('\nReview volume extracted!')
    return df

product information

In [14]:
# def product_information(df,meta_table):
#     meta= read_data_from_mysql(meta_table)
#     df = pd.merge(df, meta[['asin', 'title', 'brand', 'description', 'price']],
#                      on='asin', how='inner')
#     print('\nProduct information extracted!')
#     return df

**cell phones**

In [15]:
file_name='cell_phones_reviews_filter'
reviews_cell_phones = read_data_from_mysql(file_name)
with open('feature_names.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write("%s\n" % file_name)
with open('topics_keywords.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write("%s\n" % file_name)
    f.close()

reviews_cell_phones.head()

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18686/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load cell_phones_reviews_filter from MySQL!


,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score
0,A1EWN6KZ4HMLT7,011040047X,1.0,2012-08-14,"The case pictured is a soft violet color, but ...",2,2,1.0
1,A180NNPPKWCCU0,0110400550,5.0,2013-07-18,case fits perfectly and I always gets complime...,3,3,1.0
2,A3HVRXV0LVJN7,0110400550,5.0,2013-01-13,Best phone case ever . Everywhere I go I get a...,4,4,1.0
3,A1BJGDS0L1IO6I,0110400550,1.0,2013-01-30,ITEM NOT SENT from Blue Top Company in Hong Ko...,0,3,0.0
4,A2UCUB1VC1SDJK,0110400550,5.0,2012-07-14,This case is extremely durabl;e. I've dropped ...,6,6,1.0


In [16]:
len(reviews_cell_phones)

356664

In [18]:
reviews_cell_phones=clean_text(reviews_cell_phones)
reviews_cell_phones=review_length(reviews_cell_phones)
reviews_cell_phones=num_sentences(reviews_cell_phones)
reviews_cell_phones=avg_sentence_length(reviews_cell_phones)
reviews_cell_phones=subjective(reviews_cell_phones)
reviews_cell_phones=readability(reviews_cell_phones)
reviews_cell_phones=timeline(reviews_cell_phones)
# LDA and TF-ID
# reviews_cell_phones=Lexical(reviews_cell_phones)
reviews_cell_phones=topics(reviews_cell_phones)
# context features:
reviews_cell_phones=review_volume(reviews_cell_phones)
# reviews_cell_phones=product_information(reviews_cell_phones,'meta_cell_phones')


Text cleaned!

Review length extracted!

Number of sentences extracted!

Average sentence length extracted!

Subjective extracted!

Readability extracted!

Timeline extracted!
tfidf get
nmf_model get
nmf_topic_matrix get
nmf_topic_matrix normalize

Topics extracted:
df[topic] get: 0    1
1    1
2    1
3    6
4    1
Name: topic, dtype: int64

Review volume extracted!


In [19]:
pd.set_option('display.max_rows', None)  # 自动调整显示宽度，以适应数据
reviews_cell_phones.head()

,reviewerID,asin,overall,reviewText,helpful_vote,total_vote,helpfulness_score,cleaned_review,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline,topic,review_volume
0,A1EWN6KZ4HMLT7,011040047X,1.0,"The case pictured is a soft violet color, but ...",2,2,1.0,case pictur soft violet color case cover recei...,38,2,19.000000,0.583778,0.0,716,1,1
1,A180NNPPKWCCU0,0110400550,5.0,case fits perfectly and I always gets complime...,3,3,1.0,case fit perfectli alway get compliment crack ...,23,2,11.500000,1.000000,0.0,378,1,12
2,A3HVRXV0LVJN7,0110400550,5.0,Best phone case ever . Everywhere I go I get a...,4,4,1.0,best phone case ever everywher get ton complim...,25,3,8.333333,0.650000,8.8,564,1,12
3,A1BJGDS0L1IO6I,0110400550,1.0,ITEM NOT SENT from Blue Top Company in Hong Ko...,0,3,0.0,item sent blue top compani hong kong two month...,34,4,8.500000,0.533333,7.2,547,6,12
4,A2UCUB1VC1SDJK,0110400550,5.0,This case is extremely durabl;e. I've dropped ...,6,6,1.0,case extrem durabl drop phone atleast ten time...,70,3,23.333333,0.531481,7.2,747,1,12


In [20]:
reviews_cell_phones.dtypes

reviewerID              object
asin                    object
overall                float64
reviewText              object
helpful_vote             int64
total_vote               int64
helpfulness_score      float64
cleaned_review          object
review_length            int64
num_sentences            int64
avg_sentence_length    float64
subjectivity           float64
readability            float64
timeline                 int64
topic                    int64
review_volume            int64
dtype: object

In [21]:
len(reviews_cell_phones)

356664

In [22]:
saved_mysql(reviews_cell_phones,'features_cell_phones')

Table 'features_cell_phones' exists: False
Table 'features_cell_phones' does not exist. Creating new table...
Table 'features_cell_phones' created.
Finished saving features_cell_phones to MySQL!


**electronics**

In [23]:
file_name='electronics_reviews_filter'
reviews_electronics = read_data_from_mysql(file_name)
with open('feature_names.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write("%s\n" % file_name)
with open('topics_keywords.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write("%s\n" % file_name)
    f.close()
reviews_electronics.head()

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18686/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load electronics_reviews_filter from MySQL!


,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score
0,A1GI0U4ZRJA8WN,0439886341,1.0,2012-04-18,Do not waste your money on this thing it is te...,4,4,1.000000
1,AMO214LNFCEI4,0528881469,1.0,2010-11-25,"I'm a professional OTR truck driver, and I bou...",12,15,0.800000
2,A28B1G1MSJ6OO1,0528881469,4.0,2010-07-25,This is a great truck GPS. I've tried others a...,21,22,0.954545
3,A3N7T0DY83Y4IG,0528881469,3.0,2010-09-09,"Well, what can I say. I've had this unit in m...",43,45,0.955556
4,A1H8PY3QHMQQA0,0528881469,2.0,2010-11-24,"Not going to write a long review, even thought...",9,10,0.900000


In [24]:
reviews_electronics=review_length(reviews_electronics)
reviews_electronics=reviews_electronics.sort_values(by=['review_length'],ascending=False)
reviews_electronics=num_sentences(reviews_electronics)
reviews_electronics=avg_sentence_length(reviews_electronics)
reviews_electronics=subjective(reviews_electronics)
reviews_electronics=readability(reviews_electronics)
reviews_electronics=timeline(reviews_electronics)
# LDA and TF-IDF
reviews_electronics=clean_text(reviews_electronics)
# reviews_electronics=Lexical(reviews_electronics)
reviews_electronics=topics(reviews_electronics)
# context features:
reviews_electronics=review_volume(reviews_electronics)
# reviews_electronics=product_information(reviews_electronics,'meta_electronics')


Review length extracted!

Number of sentences extracted!

Average sentence length extracted!

Subjective extracted!

Readability extracted!

Timeline extracted!

Text cleaned!
tfidf get
nmf_model get
nmf_topic_matrix get
nmf_topic_matrix normalize

Topics extracted:
df[topic] get: 581944     2
1016398    9
1180650    2
458434     2
845961     9
Name: topic, dtype: int64

Review volume extracted!


In [25]:
reviews_electronics.head()

,reviewerID,asin,overall,reviewText,helpful_vote,total_vote,helpfulness_score,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline,cleaned_review,topic,review_volume
581944,ADPKRJ2BSFJ0I,B003WJR68U,5.0,I am updating this review constantly so check ...,533,545,0.977982,6803,382,17.808901,0.447258,8.2,1291,updat review constantli check back everi often...,2,115
1016398,A202C0YPF6B1JE,B0078GCLUG,5.0,I'll throw out some important information firs...,16,21,0.761905,6721,289,23.256055,0.430560,9.8,566,throw import inform first packag includ panaso...,9,47
1180650,A3CP1WGNTOSKDA,B0096W1P6G,5.0,This camera is not for novices and neither is ...,232,243,0.954733,6572,320,20.537500,0.475533,10.4,597,camera novic neither review fasten seatbelt a9...,2,44
458434,A2VLF9FE68LFX6,B0035LCZSG,4.0,I have been researching $500-range camcorders ...,15,15,1.000000,6538,223,29.318386,0.526734,11.0,1211,research 500rang camcord week read everi revie...,2,59
845961,A21OCIA8RSJJFC,B005CNR7B0,5.0,"Comparison Review: ATH-A9X, ATH-A900X, AKG K55...",12,18,0.666667,6537,191,34.225131,0.488034,12.0,536,comparison review atha9x atha900x akg k550 aud...,9,65


In [26]:
reviews_electronics.dtypes

reviewerID              object
asin                    object
overall                float64
reviewText              object
helpful_vote             int64
total_vote               int64
helpfulness_score      float64
review_length            int64
num_sentences            int64
avg_sentence_length    float64
subjectivity           float64
readability            float64
timeline                 int64
cleaned_review          object
topic                    int64
review_volume            int64
dtype: object

In [27]:
len(reviews_electronics)

1291534

In [28]:
saved_mysql(reviews_electronics,'features_electronics')

Table 'features_electronics' exists: False
Table 'features_electronics' does not exist. Creating new table...
Table 'features_electronics' created.
Finished saving features_electronics to MySQL!


**video games**

In [29]:
file_name='video_games_reviews_filter'
reviews_video_games = read_data_from_mysql(file_name)
with open('results/feature_names.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write(file_name)
    f.write('\n')
with open('results/topics_keywords.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write(file_name)
    f.write('\n')
reviews_video_games.head()

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18686/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load video_games_reviews_filter from MySQL!


,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score
0,A182S3ANC0W7DL,0439342260,4.0,2012-12-19,I am an Ice Cream Truck Vendor (I lease out 20...,2,2,1.000000
1,APDCEJMFDO2YT,0439394422,5.0,2010-09-27,"Such fast shipping, games is such great condit...",2,3,0.666667
2,A2H3TQWU51W1WE,043940133X,5.0,2011-03-07,My son who hates doing math loves to play this...,3,3,1.000000
3,A3KO10N2ODLHBR,043940133X,5.0,2009-11-16,My seven year old has had a lot of fun with th...,8,8,1.000000
4,A1TBUSGCBTXWFC,043940133X,1.0,2009-10-25,Came across this accidentally and took it home...,3,7,0.428571


In [30]:
reviews_video_games=review_length(reviews_video_games)
reviews_video_games=reviews_video_games.sort_values(by=['review_length'],ascending=False)
reviews_video_games=num_sentences(reviews_video_games)
reviews_video_games=avg_sentence_length(reviews_video_games)
reviews_video_games=subjective(reviews_video_games)
reviews_video_games=readability(reviews_video_games)
reviews_video_games=timeline(reviews_video_games)
# # # LDA and TF-IDF
reviews_video_games=clean_text(reviews_video_games)
# reviews_video_games=Lexical(reviews_video_games)
# #
reviews_video_games=topics(reviews_video_games)
# context features:
reviews_video_games=review_volume(reviews_video_games)
# reviews_video_games=product_information(reviews_video_games,'meta_video_games')


Review length extracted!

Number of sentences extracted!

Average sentence length extracted!

Subjective extracted!

Readability extracted!

Timeline extracted!

Text cleaned!
tfidf get
nmf_model get
nmf_topic_matrix get
nmf_topic_matrix normalize

Topics extracted:
df[topic] get: 35781     7
107310    7
137414    7
68182     7
70681     7
Name: topic, dtype: int64

Review volume extracted!


In [31]:
reviews_video_games.head()

,reviewerID,asin,overall,reviewText,helpful_vote,total_vote,helpfulness_score,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline,cleaned_review,topic,review_volume
35781,A2BN7W1SYPOKXY,B00178630A,1.0,I've played Diablo 1 and 2. Now that I have ha...,15,16,0.937500,7061,369,19.135501,0.503208,8.3,725,play diablo 300 hour game diablo say game good...,7,3473
107310,A2RMHF5YL8TE27,B0038TT8QM,5.0,"So here is the deal...I am reviewing the game,...",22,46,0.478261,6962,301,23.129568,0.493450,9.9,1406,deal review game steam account annoy anyon per...,7,1077
137414,ALUXVG3MZV3SC,B0045Y0ORG,4.0,"""Dive to the Titanic"" Review and NotesTo make ...",7,9,0.777778,6890,308,22.370130,0.413449,8.7,1164,dive titan review notesto make steamtml studio...,7,2
68182,AOMN8G5VQP74Z,B002BSA1C6,3.0,This is a review of the whole game - both sing...,1,3,0.333333,6555,204,32.132353,0.481855,10.1,1341,review whole game singl multiplay almost year ...,7,359
70681,A2582KMXLK2P06,B002BSC54I,4.0,"First of all, I have something important to sh...",2,5,0.400000,6532,264,24.742424,0.465303,11.0,767,first someth import share yall last zelda game...,7,247


In [32]:
len(reviews_video_games)

183731

In [33]:
reviews_video_games.dtypes

reviewerID              object
asin                    object
overall                float64
reviewText              object
helpful_vote             int64
total_vote               int64
helpfulness_score      float64
review_length            int64
num_sentences            int64
avg_sentence_length    float64
subjectivity           float64
readability            float64
timeline                 int64
cleaned_review          object
topic                    int64
review_volume            int64
dtype: object

In [34]:
saved_mysql(reviews_video_games,'features_video_games')

Table 'features_video_games' exists: False
Table 'features_video_games' does not exist. Creating new table...
Table 'features_video_games' created.
Finished saving features_video_games to MySQL!


**books**

In [15]:
file_name='books_reviews_filter'
with open('results/feature_names.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write(file_name)
    f.write('\n')
with open('results/topics_keywords.txt', 'a') as f:
    f.write("\n==============================\n" )
    f.write(file_name)
    f.write('\n')
reviews_books = read_data_from_mysql(file_name)
reviews_books.head()

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_13126/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load books_reviews_filter from MySQL!


,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score
0,A23PISU0ZLW71C,0000029831,5.0,2014-02-24,I thought this book was great before buying it...,5,5,1.000000
1,AF7CSSGV93RXN,000100039X,5.0,2012-03-12,I first read The Prophet by Kahlil Gibran over...,5,6,0.833333
2,A37473RZR4WOHK,000100039X,5.0,2013-03-14,While it is certainly true that Gibran was no ...,5,6,0.833333
3,A3IS4WGMFR4X65,000100039X,5.0,2009-10-27,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000
4,A39FW2WG64TQGO,000100039X,2.0,2012-06-03,I'm afraid that if I write too much in this re...,2,14,0.142857


In [16]:
reviews_books=review_length(reviews_books)
reviews_books=num_sentences(reviews_books)
reviews_books=avg_sentence_length(reviews_books)
reviews_books=subjective(reviews_books)
reviews_books.head()


Review length extracted!

Number of sentences extracted!

Average sentence length extracted!

Subjective extracted!


,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score,review_length,num_sentences,avg_sentence_length,subjectivity
0,A23PISU0ZLW71C,0000029831,5.0,2014-02-24,I thought this book was great before buying it...,5,5,1.000000,110,4,27.500000,0.619324
1,AF7CSSGV93RXN,000100039X,5.0,2012-03-12,I first read The Prophet by Kahlil Gibran over...,5,6,0.833333,82,5,16.400000,0.546627
2,A37473RZR4WOHK,000100039X,5.0,2013-03-14,While it is certainly true that Gibran was no ...,5,6,0.833333,67,5,13.400000,0.511111
3,A3IS4WGMFR4X65,000100039X,5.0,2009-10-27,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000,442,12,36.833333,0.464232
4,A39FW2WG64TQGO,000100039X,2.0,2012-06-03,I'm afraid that if I write too much in this re...,2,14,0.142857,210,6,35.000000,0.498457


In [17]:
reviews_books=readability(reviews_books)
reviews_books=timeline(reviews_books)
reviews_books.head()


Readability extracted!

Timeline extracted!


,reviewerID,asin,overall,reviewText,helpful_vote,total_vote,helpfulness_score,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline
0,A23PISU0ZLW71C,0000029831,5.0,I thought this book was great before buying it...,5,5,1.000000,110,4,27.500000,0.619324,9.3,157
1,AF7CSSGV93RXN,000100039X,5.0,I first read The Prophet by Kahlil Gibran over...,5,6,0.833333,82,5,16.400000,0.546627,8.8,871
2,A37473RZR4WOHK,000100039X,5.0,While it is certainly true that Gibran was no ...,5,6,0.833333,67,5,13.400000,0.511111,13.8,504
3,A3IS4WGMFR4X65,000100039X,5.0,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000,442,12,36.833333,0.464232,12.0,1738
4,A39FW2WG64TQGO,000100039X,2.0,I'm afraid that if I write too much in this re...,2,14,0.142857,210,6,35.000000,0.498457,10.1,788


In [18]:
# LDA and TF-IDF
reviews_books=clean_text(reviews_books)


Text cleaned!


In [20]:
reviews_books=topics(reviews_books)

tfidf get
nmf_model get
nmf_topic_matrix get
nmf_topic_matrix normalize

Topics extracted:
df[topic] get: 0    5
1    9
2    4
3    4
4    9
Name: topic, dtype: int64


In [21]:
reviews_books.head()

,reviewerID,asin,overall,reviewText,helpful_vote,total_vote,helpfulness_score,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline,cleaned_review,topic
0,A23PISU0ZLW71C,0000029831,5.0,I thought this book was great before buying it...,5,5,1.000000,110,4,27.500000,0.619324,9.3,157,thought book great buy buyer remors realli lik...,5
1,AF7CSSGV93RXN,000100039X,5.0,I first read The Prophet by Kahlil Gibran over...,5,6,0.833333,82,5,16.400000,0.546627,8.8,871,first read prophet kahlil gibran year ago read...,9
2,A37473RZR4WOHK,000100039X,5.0,While it is certainly true that Gibran was no ...,5,6,0.833333,67,5,13.400000,0.511111,13.8,504,certainli true gibran christian necessarili tr...,4
3,A3IS4WGMFR4X65,000100039X,5.0,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000,442,12,36.833333,0.464232,12.0,1738,prophet kahlil gibran best known work western ...,4
4,A39FW2WG64TQGO,000100039X,2.0,I'm afraid that if I write too much in this re...,2,14,0.142857,210,6,35.000000,0.498457,10.1,788,afraid write much review longer actual book he...,9


In [22]:
# reviewsx_books=Lexical(reviews_books)

In [23]:
# context features:
reviews_books=review_volume(reviews_books)
# reviews_books=product_information(reviews_books,'meta_books')


Review volume extracted!


In [24]:
reviews_books.head()

,reviewerID,asin,overall,reviewText,helpful_vote,total_vote,helpfulness_score,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline,cleaned_review,topic,review_volume
0,A23PISU0ZLW71C,0000029831,5.0,I thought this book was great before buying it...,5,5,1.000000,110,4,27.500000,0.619324,9.3,157,thought book great buy buyer remors realli lik...,5,1
1,AF7CSSGV93RXN,000100039X,5.0,I first read The Prophet by Kahlil Gibran over...,5,6,0.833333,82,5,16.400000,0.546627,8.8,871,first read prophet kahlil gibran year ago read...,9,18
2,A37473RZR4WOHK,000100039X,5.0,While it is certainly true that Gibran was no ...,5,6,0.833333,67,5,13.400000,0.511111,13.8,504,certainli true gibran christian necessarili tr...,4,18
3,A3IS4WGMFR4X65,000100039X,5.0,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000,442,12,36.833333,0.464232,12.0,1738,prophet kahlil gibran best known work western ...,4,18
4,A39FW2WG64TQGO,000100039X,2.0,I'm afraid that if I write too much in this re...,2,14,0.142857,210,6,35.000000,0.498457,10.1,788,afraid write much review longer actual book he...,9,18


In [25]:
reviews_books.dtypes

reviewerID              object
asin                    object
overall                float64
reviewText              object
helpful_vote             int64
total_vote               int64
helpfulness_score      float64
review_length            int64
num_sentences            int64
avg_sentence_length    float64
subjectivity           float64
readability            float64
timeline                 int64
cleaned_review          object
topic                    int64
review_volume            int64
dtype: object

In [26]:
len(reviews_books)

1623922

In [27]:
saved_mysql(reviews_books,'features_books')

Table 'features_books' exists: False
Table 'features_books' does not exist. Creating new table...
Table 'features_books' created.
Finished saving features_books to MySQL!


In [30]:
reviews_tables = ['reviews_cell_phones', 'reviews_electronics', 'reviews_video_games', 'reviews_books']
meta_tables = ['meta_cell_phones', 'meta_electronics', 'meta_video_games', 'meta_books']

# Feature Selection

In [5]:
features_cell_phones=read_data_from_mysql('features_cell_phones')
features_cell_phones.head()

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18674/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_cell_phones from MySQL!


,helpfulness_score,reviewText,cleaned_review,overall,review_length,num_sentences,avg_sentence_length,subjectivity,readability,timeline,topic,review_volume
0,1.0,"The case pictured is a soft violet color, but ...",case pictur soft violet color case cover recei...,1.0,38,2,19.00000,0.583778,0.0,716,1,1
1,1.0,case fits perfectly and I always gets complime...,case fit perfectli alway get compliment crack ...,5.0,23,2,11.50000,1.000000,0.0,378,1,12
2,1.0,Best phone case ever . Everywhere I go I get a...,best phone case ever everywher get ton complim...,5.0,25,3,8.33333,0.650000,8.8,564,1,12
3,0.0,ITEM NOT SENT from Blue Top Company in Hong Ko...,item sent blue top compani hong kong two month...,1.0,34,4,8.50000,0.533333,7.2,547,6,12
4,1.0,This case is extremely durabl;e. I've dropped ...,case extrem durabl drop phone atleast ten time...,5.0,70,3,23.33330,0.531482,7.2,747,1,12


In [6]:
features_electronics=read_data_from_mysql('features_electronics')

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18674/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_electronics from MySQL!


In [7]:
features_video_games=read_data_from_mysql('features_video_games')

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18674/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_video_games from MySQL!


In [8]:
features_books=read_data_from_mysql('features_books')

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_18674/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_books from MySQL!


**Random Forest**

In [4]:
from sklearn.ensemble import  RandomForestRegressor

In [11]:
def random_forest(df):
    # 将特征列和目标变量分离
    X = df.drop('reviewText', axis=1)
    X = X.drop('cleaned_review', axis=1)
    X = X.drop('helpfulness_score', axis=1)

    y = df['helpfulness_score']

    # 初始化随机森林回归模型
    rf_model = RandomForestRegressor(random_state=42)

    # 训练随机森林模型
    rf_model.fit(X, y)

    # 获取特征的重要性得分
    feature_importance = rf_model.feature_importances_

    # 创建一个包含特征和对应重要性得分的DataFrame
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

    # 将特征按照重要性得分降序排序
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # 输出特征重要性排名,包含他们的重要性得分
    print(feature_importance_df)


In [12]:
random_forest(features_cell_phones)

               Feature  Importance
6             timeline    0.185235
4         subjectivity    0.159374
8        review_volume    0.142894
1        review_length    0.119980
3  avg_sentence_length    0.113795
0              overall    0.108185
5          readability    0.077281
7                topic    0.062649
2        num_sentences    0.030607


In [13]:
random_forest(features_electronics)

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_25001/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_electronics from MySQL!
               Feature  Importance
6             timeline    0.177215
4         subjectivity    0.158457
8        review_volume    0.144160
1        review_length    0.142677
0              overall    0.109479
3  avg_sentence_length    0.103602
5          readability    0.080917
7                topic    0.057192
2        num_sentences    0.026300


In [14]:

random_forest(features_video_games)

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_25001/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_video_games from MySQL!
               Feature  Importance
8        review_volume    0.200529
1        review_length    0.174542
6             timeline    0.164762
4         subjectivity    0.132785
3  avg_sentence_length    0.099103
5          readability    0.072832
0              overall    0.069694
7                topic    0.055685
2        num_sentences    0.030067


In [15]:

random_forest(features_books)

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_25001/2787366574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load features_books from MySQL!
               Feature  Importance
8        review_volume    0.166395
1        review_length    0.165370
0              overall    0.160028
6             timeline    0.154993
4         subjectivity    0.129131
3  avg_sentence_length    0.084343
5          readability    0.069275
7                topic    0.047142
2        num_sentences    0.023322


**Correlation**

In [15]:
def correlation(df):
    # 选择需要进行相关性分析的特征列
    features = ['overall', 'review_length', 'num_sentences', 'avg_sentence_length',
                'subjectivity', 'readability', 'timeline','topic','review_volume','helpfulness_score']

    # 计算特征与目标变量(helpfulness_score)之间的相关系数
    correlations_spc = df[features].corr(method='spearman')
    correlations_spc = correlations_spc.round(decimals=3)

    # 提取特征与目标变量(helpfulness_score)的相关系数
    correlations_with_target_spc = correlations_spc['helpfulness_score'].drop('helpfulness_score').abs()
    # 输出特征与目标变量(helpfulness_score)的相关系数
    print('\n=========SPC:========\n',correlations_with_target_spc,'\n')
    print('--------------------------------------------------')



        # 计算特征与目标变量(helpfulness_score)之间的相关系数
    correlations_pcc = df[features].corr(method='pearson')
    correlations_pcc = correlations_pcc.round(decimals=3)

    # 提取特征与目标变量(helpfulness_score)的相关系数
    correlations_with_target_pcc = correlations_pcc['helpfulness_score'].drop('helpfulness_score').abs()

    # 输出特征与目标变量(helpfulness_score)的相关系数
    print('\n=========PCC=========:\n',correlations_with_target_pcc)


In [16]:
correlation(features_cell_phones)


=========SPC:========
 overall                0.287
review_length          0.100
num_sentences          0.113
avg_sentence_length    0.004
subjectivity           0.064
readability            0.085
timeline               0.104
topic                  0.016
review_volume          0.215
Name: helpfulness_score, dtype: float64 

--------------------------------------------------

=========PCC=========:
 overall                0.302
review_length          0.111
num_sentences          0.111
avg_sentence_length    0.004
subjectivity           0.076
readability            0.121
timeline               0.104
topic                  0.014
review_volume          0.097
Name: helpfulness_score, dtype: float64


In [17]:
correlation(features_electronics)


=========SPC:========
 overall                0.299
review_length          0.193
num_sentences          0.182
avg_sentence_length    0.066
subjectivity           0.044
readability            0.147
timeline               0.097
topic                  0.014
review_volume          0.156
Name: helpfulness_score, dtype: float64 

--------------------------------------------------

=========PCC=========:
 overall                0.303
review_length          0.169
num_sentences          0.163
avg_sentence_length    0.027
subjectivity           0.059
readability            0.191
timeline               0.113
topic                  0.030
review_volume          0.064
Name: helpfulness_score, dtype: float64


In [18]:
correlation(features_video_games)


=========SPC:========
 overall                0.209
review_length          0.282
num_sentences          0.259
avg_sentence_length    0.117
subjectivity           0.004
readability            0.233
timeline               0.147
topic                  0.060
review_volume          0.180
Name: helpfulness_score, dtype: float64 

--------------------------------------------------

=========PCC=========:
 overall                0.218
review_length          0.191
num_sentences          0.178
avg_sentence_length    0.038
subjectivity           0.020
readability            0.251
timeline               0.153
topic                  0.070
review_volume          0.058
Name: helpfulness_score, dtype: float64


In [19]:
# overall,review_length,num_sentences,avg_sentence_length,subjectivity

In [20]:
correlation(features_books)


=========SPC:========
 overall                0.401
review_length          0.278
num_sentences          0.245
avg_sentence_length    0.166
subjectivity           0.017
readability            0.218
timeline               0.021
topic                  0.077
review_volume          0.233
Name: helpfulness_score, dtype: float64 

--------------------------------------------------

=========PCC=========:
 overall                0.396
review_length          0.169
num_sentences          0.150
avg_sentence_length    0.095
subjectivity           0.023
readability            0.273
timeline               0.027
topic                  0.085
review_volume          0.086
Name: helpfulness_score, dtype: float64
